In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import pandas as pd

import numpy as np 

import create_model
import pickle
import time 
import copy
import random
import data_formatting


def validate(train):
    
    if train == True:
        
        model = train_model
        mode = 'TRAIN'
        
    else:
        mode = 'DEV'
        model = dev_model

    encoder, decoder, predicted = session.run([model.encoder_inputs, model.decoder_targets, model.decoder_pred_train])            

    print ('Current mode:%s' % mode)
    for i, (e_in, dt_targ, dt_pred) in enumerate(zip( encoder, decoder, predicted)):

        print('  sample {}:'.format(i + 1))
        #print('    enc input           > {}'.format(e_in))
        print('    enc input           > {}'.format(data_formatting.decodeSent(e_in, inv_map)))

        #print('    dec input           > {}'.format(dt_targ))
        print('    dec input           > {}'.format(data_formatting.decodeSent(dt_targ, inv_map)))

        #print('    dec train predicted > {}'.format(dt_pred))
        print('    dec train predicted > {}'.format(data_formatting.decodeSent(dt_pred, inv_map)))

        if i >= 0: break

In [2]:
#dataset = 'enron'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')

dataset = 'twitter'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl', 'rb'))
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl')

In [3]:
inv_map = data_formatting.createInvMap(vocab_dict)

In [4]:
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['alpha_Pair_1_encoding'] = df_all['alpha_Pair_1_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['Index'] = df_all.index.values

In [5]:
df_all_train = df_all.sample(frac=0.97, random_state=1)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

df_all_test = df_all_dev.sample(frac=0.10, random_state=1)

df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

In [6]:
print (df_all.shape[0], df_all_train.shape[0],  df_all_dev.shape[0], df_all_test.shape[0], len(vocab_dict))

17690 17159 478 53 17322


In [7]:
train_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

dev_data = data_formatting.prepare_train_batch(df_all_dev['alpha_Pair_0_encoding'].values, 
                                                    df_all_dev['alpha_Pair_1_encoding'].values)

test_data = data_formatting.prepare_train_batch(df_all_test['alpha_Pair_0_encoding'].values, 
                                                    df_all_test['alpha_Pair_1_encoding'].values)

In [8]:
train_model_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':10, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

In [9]:
dev_model_params = train_model_params
dev_model_params['encoder_input_keep'] = 1
dev_model_params['encoder_output_keep'] = 1
dev_model_params['decoder_input_keep'] = 1
dev_model_params['decoder_output_keep'] = 1

In [10]:
training_params = { 'vocab_lower':3, 'vocab_upper':train_model_params['vocab_size']-1, 
                    'n_epochs':500000}

In [11]:
#enc input           > ['i', 'll', 'be', 'there', 'tomorrow', '<EOS>']
#dec input           > ['last', 'game', 'of', 'the', 'regular', 'season', '<EOS>']

In [12]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops

In [39]:
tf.reset_default_graph()

inf_model_1_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':1, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#inf_model_2_params = {'n_cells':2, 'num_layers':1, 'embedding_size':5, 
#            'vocab_size':7,
#          'minibatch_size':64, 'n_threads':128,
#          'beam_width':3, 'limit_decode_steps': None,
#          'encoder_input_keep':1, 'decoder_input_keep':1,
#          'encoder_output_keep':1, 'decoder_output_keep':1,
#         }


inf_model_2_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':7, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#with tf.variable_scope('training_model'):

#    inf_model_1 = create_model.Model(inf_model_1_params, 'infer', None)
    
#with tf.variable_scope('training_model'):

#    inf_model_2 = create_model.Model(inf_model_2_params, 'infer', None)   

with tf.variable_scope('training_model'):

    inf_train = create_model.Model(inf_model_2_params, 'debug', None)   

In [40]:
#names_to_vars_inf = [[v.op.name, v] for v in tf.all_variables()]

In [41]:
names_to_vars = [[v.op.name, v] for v in tf.all_variables()]

Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Please use tf.global_variables instead.


In [42]:
#[i[0] for i in names_to_vars_inf if 'forget' in i[0]]

In [43]:
[i[0] for i in names_to_vars if 'forget' in i[0]]

['training_model/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/rnn/multi_rnn_cell/cell_1/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/rnn/multi_rnn_cell/cell_1/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/decoder/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/decoder/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/forget/beta',
 'training_model/decoder/multi_rnn_cell/cell_1/Attention_Wrapper/layer_norm_basic_lstm_cell/forget/gamma',
 'training_model/decoder/multi_rnn_cell/cell_1/Attention_Wrapper/layer_norm_basic_lstm_cell/forget/beta']

In [44]:
df_all_dev.values[12]

array([ list(['and', 'it', 'be', 'the', 'one', 'going', 'for', 'one', 'stop', '<EOS>']),
       list(['and', 'the', 'bus', 'omg', '<EOS>']),
       list([8378, 13635, 14926, 7441, 6200, 9187, 434, 6200, 3425, 1]),
       list([8378, 7441, 11483, 11526, 1]), 353842], dtype=object)

In [45]:
input_user = data_formatting.encodeSent(['thank', 'you'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run(
        inf_train.final_outputs, 
        #inf_model_2.decoder_pred_decode,
        feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [46]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

array([[['jodi', 'now', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>',
         '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>',
         '<EOS>'],
        ['-0.6924710869789124', '-2.4161858558654785',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787', '-2.4267899990081787',
         '-2.4267899990081787']],

       [['congratulation', 'derek', '<EOS>', '<EOS>', '<EOS>', '<EOS>',
         '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>',
         '<EOS>', '<EOS>'],
        ['-1.7698712348937988', '-2.4260313510894775',
         '-2.969587802886963', '-2.969587802886963', '-2.969587802886963',
         '-2.969587802886963', '-2.969587802886963', '-2.969587802886963',
         '-2.969587802886963', '-2.96958780

In [31]:
#data_formatting.decodeSent(q_0.beam_search_decoder_output.predicted_ids.T[0][0], inv_map)

In [18]:
batch_size = inf_model_2.inference_decoder._batch_size
beam_width = inf_model_2.inference_decoder._beam_width
end_token = inf_model_2.inference_decoder._end_token
length_penalty_weight = inf_model_2.inference_decoder._length_penalty_weight

In [19]:
name= None

In [20]:
time = 0

In [48]:
finished, first_inputs, initial_state = inf_model_2.inference_decoder.initialize()

In [ ]:
inf_model_2.

In [49]:
first_inputs, initial_state

(<tf.Tensor 'training_model/embedding_lookup_1:0' shape=(?, 3, 512) dtype=float32>,
 BeamSearchDecoderState(cell_state=(LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_1:0' shape=(?, 3, 256) dtype=float32>), AttentionWrapperState(cell_state=LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape_2:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_3:0' shape=(?, 3, 256) dtype=float32>), attention=<tf.Tensor 'training_model/Reshape_4:0' shape=(?, 3, 256) dtype=float32>, time=<tf.Tensor 'training_model/AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'training_model/Reshape_5:0' shape=(?, 3, ?) dtype=float32>, alignment_history=())), log_probs=<tf.Tensor 'zeros_2:0' shape=(?, 3) dtype=float32>, finished=<tf.Tensor 'training_model/zeros:0' shape=(?, 3) dtype=bool>, lengths=<tf.Tensor 'zeros_3:0' shape=(?, 3) dtype=int32>))

In [50]:
#first_inputs
#These are the word embeddings of the initial token (which for initialize should be from the encoder?)
#Shape is [batch_size, embedding_size]
#initial_state
#This is the initial state of the decoder, a tuple consisting of 
#1. the LSTM cell and hiddens states
#2. The attention states

In [51]:
#When we pass time !=0 to inference decoder, even with initial step results we get non-zero parent ids

#beam_search_output, beam_search_state, next_inputs, finished = \
#                                                inf_model_2.inference_decoder.step(time, first_inputs, initial_state)

In [52]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state, batch_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [53]:
q_0[0].shape

(1, 3, 512)

In [54]:
#with ops.name_scope(name, "BeamSearchDecoderStep", (time, first_inputs, initial_state)):
    
cell_state = initial_state.cell_state

first_inputs = nest.map_structure(lambda inp: inf_model_2.inference_decoder._merge_batch_beams(inp, s=inp.shape[2:]), 
                                  first_inputs)

cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_merge_batch_beams, cell_state, 
                                inf_model_2.inference_decoder._cell.state_size)

In [55]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [56]:
q_0[0].shape

(3, 512)

In [17]:
##This goes through _cell first
#The output of cell_outputs is also present in the next_cell_state, the cell_outputs is the attention

#The cell_state includes, the cell and hidden states of the LSTM, the attention states, the alignment history, and timestep, 
#what does alignment history and attention states really mean? need to internalize...

cell_outputs, next_cell_state = inf_model_2.inference_decoder._cell(first_inputs, cell_state)

NameError: name 'inf_model_2' is not defined

In [16]:
cell_outputs

NameError: name 'cell_outputs' is not defined

In [58]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([cell_outputs, next_cell_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user, input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user), len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [59]:
q_0[0].shape

(6, 256)

In [60]:
#These functions convert the shape of the cell outputs and states from [batch_size, beam_width, embedding_size]
#to [batch_size*beam_width, embedding_size], essentially, reducing a dimensionality by flattening the nested structure
#the map structure fn is used because cell state is a complex tuple structure, 
#outputs is a simpe structure (should be able to just call flatten?)
cell_outputs = nest.map_structure(lambda out: inf_model_2.inference_decoder._split_batch_beams(out, out.shape[1:]), 
                                  cell_outputs)

next_cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_split_batch_beams, next_cell_state, 
                                     inf_model_2.inference_decoder._cell.state_size)

if inf_model_2.inference_decoder._output_layer is not None:
    cell_outputs = inf_model_2.inference_decoder._output_layer(cell_outputs)

In [61]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([step_log_probs, cell_outputs, next_cell_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [62]:
q_0[1].shape

(1, 3, 17323)

In [63]:
#cell_outputs_rnn_call, next_cell_state_rnn_call = inf_model_2.inference_decoder._cell.call(first_inputs, cell_state)

#cell_outputs_rnn_call_call, next_cell_state_rnn_call_call = inf_model_2.inference_decoder._cell.call(cell_outputs_rnn_call, 
#                                                                                                     next_cell_state_rnn_call)

In [64]:
sess.close()

In [65]:
static_batch_size = tensor_util.constant_value(batch_size)

prediction_lengths = initial_state.lengths
previously_finished = initial_state.finished

In [66]:
end_token = data_formatting.EOS

In [67]:
def _check_maybe(t):
  if isinstance(t, tensor_array_ops.TensorArray):
    raise TypeError(
        "TensorArray state is not supported by BeamSearchDecoder: %s" % t.name)
  if t.shape.ndims is None:
    raise ValueError(
        "Expected tensor (%s) to have known rank, but ndims == None." % t)

def _maybe_tensor_gather_helper(gather_indices, gather_from, batch_size,
                                range_size, gather_shape):
  """Maybe applies _tensor_gather_helper.
  This applies _tensor_gather_helper when the gather_from dims is at least as
  big as the length of gather_shape. This is used in conjunction with nest so
  that we don't apply _tensor_gather_helper to inapplicable values like scalars.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
      or the original tensor if its dimensions are too small.
  """
  _check_maybe(gather_from)
  if gather_from.shape.ndims >= len(gather_shape):
    return _tensor_gather_helper(
        gather_indices=gather_indices,
        gather_from=gather_from,
        batch_size=batch_size,
        range_size=range_size,
        gather_shape=gather_shape)
  else:
    return gather_from

def _tensor_gather_helper(gather_indices, gather_from, batch_size,
                          range_size, gather_shape):
  """Helper for gathering the right indices from the tensor.
  This works by reshaping gather_from to gather_shape (e.g. [-1]) and then
  gathering from that according to the gather_indices, which are offset by
  the right amounts in order to preserve the batch order.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The input batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
  """
  range_ = array_ops.expand_dims(math_ops.range(batch_size) * range_size, 1)
  gather_indices = array_ops.reshape(gather_indices + range_, [-1])
  output = array_ops.gather(
      array_ops.reshape(gather_from, gather_shape), gather_indices)
  final_shape = array_ops.shape(gather_from)[:1 + len(gather_shape)]
  static_batch_size = tensor_util.constant_value(batch_size)
  final_static_shape = (tensor_shape.TensorShape([static_batch_size])
                        .concatenate(
                            gather_from.shape[1:1 + len(gather_shape)]))
  output = array_ops.reshape(output, final_shape)
  output.set_shape(final_static_shape)
  return output
#Have to understand how this works!
def _mask_probs(probs, eos_token, finished):
  """Masks log probabilities.
  The result is that finished beams allocate all probability mass to eos and
  unfinished beams remain unchanged.
  Args:
    probs: Log probabiltiies of shape `[batch_size, beam_width, vocab_size]`
    eos_token: An int32 id corresponding to the EOS token to allocate
      probability to.
    finished: A boolean tensor of shape `[batch_size, beam_width]` that
      specifies which
      elements in the beam are finished already.
  Returns:
    A tensor of shape `[batch_size, beam_width, vocab_size]`, where unfinished
    beams stay unchanged and finished beams are replaced with a tensor with all
    probability on the EOS token.
  """
  vocab_size = array_ops.shape(probs)[2]
  finished_mask = array_ops.expand_dims(
      math_ops.to_float(1. - math_ops.to_float(finished)), 2)
  # These examples are not finished and we leave them
  non_finished_examples = finished_mask * probs
  # All finished examples are replaced with a vector that has all
  # probability on EOS
  finished_row = array_ops.one_hot(
      eos_token,
      vocab_size,
      dtype=probs.dtype,
      on_value=0.,
      off_value=probs.dtype.min)
  finished_examples = (1. - finished_mask) * finished_row
  return finished_examples + non_finished_examples

In [68]:
logits = cell_outputs
step_log_probs = nn_ops.log_softmax(logits)

In [69]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([step_log_probs, logits, cell_outputs], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [70]:
q_0[0].shape, q_0[1].shape, q_0[2].shape

((1, 3, 17323), (1, 3, 17323), (1, 3, 17323))

In [71]:
step_log_probs

<tf.Tensor 'Reshape_22:0' shape=(?, 3, 17323) dtype=float32>

In [72]:
tf.shape(step_log_probs)[0]

<tf.Tensor 'strided_slice_23:0' shape=() dtype=int32>

In [73]:
step_log_probs = _mask_probs(step_log_probs, end_token, previously_finished)
total_probs = array_ops.expand_dims(initial_state.log_probs, 2) + step_log_probs

In [74]:
vocab_size = logits.shape[-1].value or array_ops.shape(logits)[-1]

lengths_to_add = array_ops.one_hot(
      indices=array_ops.tile(
          array_ops.reshape(end_token, [1, 1]), [batch_size, beam_width]),
      depth=vocab_size,
      on_value=0,
      off_value=1)

In [75]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([total_probs, array_ops.expand_dims(initial_state.log_probs, 2), step_log_probs], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [76]:
q_0[2][0][2] + q_0[1][0][2]

array([-33.68927383, -27.15158844, -27.81147385, ..., -16.75343132,
       -25.89280701, -28.28565216], dtype=float32)

In [77]:
q_0[0][0]

array([[-33.68927383, -27.15158844, -27.81147385, ..., -16.75342751,
        -25.8928051 , -28.28565216],
       [-33.68927002, -27.15158844, -27.81147385, ..., -16.75342941,
        -25.89280701, -28.28565216],
       [-33.68927383, -27.15158844, -27.81147385, ..., -16.75343132,
        -25.89280701, -28.28565216]], dtype=float32)

In [78]:
sess.close()

In [79]:
add_mask = (1 - math_ops.to_int32(previously_finished))

lengths_to_add = array_ops.expand_dims(add_mask, 2) * lengths_to_add

new_prediction_lengths = (lengths_to_add + array_ops.expand_dims(prediction_lengths, 2))

In [80]:
scores = total_probs

scores_shape = array_ops.shape(scores)

time = ops.convert_to_tensor(time, name="time")

scores_flat = control_flow_ops.cond(time > 0,
      lambda: array_ops.reshape(scores, [batch_size, -1]), lambda: scores[:, 0])

num_available_beam = control_flow_ops.cond(
      time > 0, lambda: math_ops.reduce_prod(scores_shape[1:]),
      lambda: math_ops.reduce_prod(scores_shape[2:]))

In [81]:
# Pick the next beams according to the specified successors function
next_beam_size = math_ops.minimum(
  ops.convert_to_tensor(beam_width, dtype=dtypes.int32, name="beam_width"),
  num_available_beam)

next_beam_scores, word_indices = nn_ops.top_k(scores_flat, k=next_beam_size)
next_beam_scores.set_shape([static_batch_size, beam_width])
word_indices.set_shape([static_batch_size, beam_width])

In [82]:
next_beam_probs = _tensor_gather_helper(
      gather_indices=word_indices,
      gather_from=scores,
      batch_size=batch_size,
      range_size=beam_width * vocab_size,
      gather_shape=[-1])

#Here, word indices represents the positions of the flattened list of beams, which goes from 1 to beam_width*vocab_size
#thus the word indices obtained are not the word indices of the 1 to N vocab but the flattened out list of logprobs from 1 to 
#beam_width*vocab_size
next_word_ids = math_ops.to_int32(word_indices % vocab_size)
#The beam_ids represent which beams these word indices (and thereby log prob values belong to), as
#we have word indices extracted out of 1 to beam_width*vocab_size
next_beam_ids = math_ops.to_int32(word_indices / vocab_size)

In [83]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([next_word_ids, next_beam_ids, scores, scores_flat, next_beam_scores, next_beam_probs,
                    word_indices, next_beam_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [84]:
q_0[2][0]

array([[-33.68927002, -27.15158653, -27.81147194, ..., -16.75342941,
        -25.89280319, -28.28565025],
       [-33.68927002, -27.15158653, -27.81147194, ..., -16.75342941,
        -25.8928051 , -28.28565025],
       [-33.68927002, -27.15158844, -27.81147385, ..., -16.75342941,
        -25.89280701, -28.28565216]], dtype=float32)

In [85]:
q_0[3][0][np.argsort(q_0[3][0])[-3:]]

array([-2.60707283, -2.0549202 , -0.85561192], dtype=float32)

In [86]:
q_0[3][0][:vocab_size], q_0[3][0][vocab_size:vocab_size*2], q_0[3][0][vocab_size*2:]

(array([-33.68927002, -27.15158653, -27.81147194, ..., -16.75342941,
        -25.89280319, -28.28565025], dtype=float32),
 array([], dtype=float32),
 array([], dtype=float32))

In [87]:
q_0[4], q_0[5]

(array([[-0.85561192, -2.0549202 , -2.60707283]], dtype=float32),
 array([[-0.85561192, -2.0549202 , -2.60707283]], dtype=float32))

In [88]:
q_0[0], q_0[1]

(array([[ 4400, 10359, 11678]]), array([[0, 0, 0]]))

In [89]:
previously_finished = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=previously_finished,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_finished = math_ops.logical_or(previously_finished, math_ops.equal(next_word_ids, end_token))

In [90]:
lengths_to_add = math_ops.to_int32(math_ops.not_equal(next_word_ids, end_token))
lengths_to_add = (1 - math_ops.to_int32(next_finished)) * lengths_to_add

In [91]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([previously_finished, next_finished, lengths_to_add], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [92]:
q_0

[array([[False, False, False]], dtype=bool),
 array([[False, False, False]], dtype=bool),
 array([[1, 1, 1]])]

In [93]:
next_prediction_len = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=initial_state.lengths,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_prediction_len += lengths_to_add

In [94]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_prediction_len],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [95]:
q_1

[array([[1, 1, 1]])]

In [96]:
next_cell_state = nest.map_structure(
      lambda gather_from: _maybe_tensor_gather_helper(
          gather_indices=next_beam_ids,
          gather_from=gather_from,
          batch_size=batch_size,
          range_size=beam_width,
          gather_shape=[batch_size * beam_width, -1]),
      next_cell_state)

In [97]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_cell_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [98]:
#These are the returns, next_state and output to the step function, but how exactly do we transpose this back and push it
#into cell? Moreover what is teh mechanics that determines when the beams become different? I.E. are no longer identical

In [99]:
next_state = tf.contrib.seq2seq.BeamSearchDecoderState(
      cell_state=next_cell_state,
      log_probs=next_beam_probs,
      lengths=next_prediction_len,
      finished=next_finished)

output = tf.contrib.seq2seq.BeamSearchDecoderOutput(
      scores=next_beam_scores,
      predicted_ids=next_word_ids,
      parent_ids=next_beam_ids)

In [100]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_state, output, next_beam_scores],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [101]:
beam_search_state = next_state
beam_search_output = output

In [102]:
finished = beam_search_state.finished

In [103]:
sample_ids = beam_search_output.predicted_ids

In [104]:
next_inputs = control_flow_ops.cond(math_ops.reduce_all(finished), 
                                    lambda: inf_model_2.inference_decoder._start_inputs,
                                    lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [105]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    #sess.run(tf.global_variables_initializer())

    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_inputs, beam_search_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]}) 

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [106]:
q_0[0].shape, q_1[0][0].shape

((1, 3), (3, 512))

In [109]:
q_1[0].shape

(1, 3, 512)

In [176]:
q_0[1][3].shape, q_1[1][3].shape

((1, 3), (1, 3))

In [177]:
q_0[1][2].shape, q_1[1][2].shape

((1, 3), (1, 3))

In [178]:
q_0[1][1].shape, q_1[1][1].shape

((1, 3), (1, 3))

In [181]:
len(q_0[1][0]), len(q_1[1][0])

(2, 2)

In [209]:
#next_inputs
#These are the chosen word ids which we have performed the embedding look up for
first_inputs = next_inputs[0]#The [0] is for batch size, should not need it

#next_states
#these are the next states of the decoder network which pass back through the process
initial_state = next_state

finished = finished

In [217]:
time =1